In [21]:
# Importación de librerías:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

# librerías de selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By

from tqdm import tqdm

In [22]:
# importar el csv con las películas obtenidas del api
df = pd.read_csv('Team1_API_extraction.csv')

# creamos una variable global para almacenar la información d los actores
lista_actores = []

# del df solo nos interesa la id_pelicula, para hacer una lista con cada idOwner de cada pelicula, y asi bsucarlas
lista_id_peliculas = list(df['idOwner'])
print(lista_id_peliculas)

['tt0052698', 'tt0053357', 'tt0053496', 'tt0052661', 'tt0053228', 'tt0053752', 'tt0051874', 'tt0053063', 'tt0053546', 'tt0052997', 'tt0052698', 'tt0053357', 'tt0053496', 'tt0052661', 'tt0053228', 'tt0053752', 'tt0051874', 'tt0053063', 'tt0053546', 'tt0052997', 'tt0052698', 'tt0053357', 'tt0053496', 'tt0052661', 'tt0053228', 'tt0053752', 'tt0051874', 'tt0053063', 'tt0053546', 'tt0052997', 'tt0052698', 'tt0053357', 'tt0053496', 'tt0052661', 'tt0053228', 'tt0053752', 'tt0051874', 'tt0053063', 'tt0053546', 'tt0052997', 'tt0053979', 'tt0053829', 'tt0054071', 'tt0054712', 'tt0054054', 'tt0054733', 'tt0054691', 'tt0054823', 'tt0054817', 'tt0054731', 'tt0053979', 'tt0053829', 'tt0054071', 'tt0054712', 'tt0054054', 'tt0054733', 'tt0054691', 'tt0054823', 'tt0054817', 'tt0054731', 'tt0053979', 'tt0053829', 'tt0054071', 'tt0054712', 'tt0054054', 'tt0054733', 'tt0054691', 'tt0054823', 'tt0054817', 'tt0054731', 'tt0053979', 'tt0053829', 'tt0054071', 'tt0054712', 'tt0054054', 'tt0054733', 'tt0054691'

# Limitamos el numero de peliculas para trabajar

In [23]:
lista_id_peliculas = lista_id_peliculas[:224]


In [24]:
# Ejecutamos el driver y vamos a la web de imdb
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/")
sleep(3)

In [26]:
#Clickamos para quitar las cookies
driver.find_element("css selector", '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
print("Hemos rechazado las cookies")

Hemos rechazado las cookies


In [27]:

#Creamos una lista vacia para almacenar los actores y actrices
list_actors_by_movie = []

# ahora recorremos cada peli en la lista de peliculas
for peli in tqdm(lista_id_peliculas):
    driver.get(f"https://www.imdb.com/title/{peli}")
    sleep(3)
    try:
        actors = driver.find_elements("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div > div.sc-bfec09a1-7.gWwKlt > a")
        if len(actors) > 10:
            actors = actors[:10]
        for actor in actors:
            actor_id = actor.get_attribute("href")
            actor_id = actor_id.replace("https://www.imdb.com/name/", "")
            actor_id = actor_id[:actor_id.find("/")]
            list_actors_by_movie.append({'id_pelicula':peli, "actor_id": actor_id})

    except:
        print("No se encontró ningún actor.")


100%|██████████| 224/224 [21:02<00:00,  5.64s/it]


In [33]:
#Convertimos los datos en un DataFrame
df_actors_movie = pd.DataFrame(list_actors_by_movie)
print(df_actors_movie)

     id_pelicula   actor_id
0      tt0052698  nm0893341
1      tt0052698  nm0590452
2      tt0052698  nm0000901
3      tt0052698  nm0197950
4      tt0052698  nm0033929
...          ...        ...
2066   tt0057948  nm0184233
2067   tt0057948  nm0603787
2068   tt0057948  nm0920639
2069   tt0057948  nm0920637
2070   tt0057948  nm0661039

[2071 rows x 2 columns]


In [29]:
#Para que no se repitan los actores y actrices se añade un unique
list_id_actors = df_actors_movie["actor_id"].unique().tolist() 

In [30]:
#Para pasar a CSV que va a conectar la tabla creada en el ejercicio 1.2. con la que finalmente necesitamos del ejercicio 1.3. 
df_actors_movie.to_csv("Movie_actor.csv",index=False)


In [31]:
# importar el csv de los actores que hemos sacado previamente
df_actors = pd.read_csv('Movie_actor.csv')

#Para que no se repitan los actores y actrices se añade un unique
list_id_actors = df_actors["actor_id"].unique().tolist() 
print(list_id_actors)


['nm0893341', 'nm0590452', 'nm0000901', 'nm0197950', 'nm0033929', 'nm0289842', 'nm0617741', 'nm0472020', 'nm0449166', 'nm0773723', 'nm0593192', 'nm0527017', 'nm0110917', 'nm0859183', 'nm0288031', 'nm0486742', 'nm0275987', 'nm0129478', 'nm0408990', 'nm0038066', 'nm0473228', 'nm0374919', 'nm0000728', 'nm0290955', 'nm0885706', 'nm0773150', 'nm0907267', 'nm0871020', 'nm0755775', 'nm0415738', 'nm0263929', 'nm0943469', 'nm0755697', 'nm0017530', 'nm0299802', 'nm0066487', 'nm0003248', 'nm0034357', 'nm0587582', 'nm0067176', 'nm0712731', 'nm0783288', 'nm0000564', 'nm0601476', 'nm0789185', 'nm0866627', 'nm0293661', 'nm0494452', 'nm0539942', 'nm0330101', 'nm0549773', 'nm0265761', 'nm0013567', 'nm0529876', 'nm0305517', 'nm2909247', 'nm0554819', 'nm0738139', 'nm0659270', 'nm0722023', 'nm0201638', 'nm0041509', 'nm0785771', 'nm0350826', 'nm0142707', 'nm0200434', 'nm0893834', 'nm0615913', 'nm0053884', 'nm0518623', 'nm0370144', 'nm0665902', 'nm0515193', 'nm0000277', 'nm0285302', 'nm0018776', 'nm0083424'

In [32]:
#Obtenemos la informacion de cada actor a traves de su id
list_info_actors = []

#Adquirimos la informacion que necesitamos

for actor_id in list_id_actors:
    print(f"Extrayendo la informacion del actor {actor_id}")
    driver.get(f"https://www.imdb.com/name/{actor_id}/?ref_=tt_ov_st")

            #Nombre 
    try:
        name = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span[1]').text

    except:
        if not name: # en caso de que no haya puntuación
            name = None
            print("We could not access the name")

     #Año de nacimiento
    year_birthday = None
    try: 
        year_element = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div[1]/span[2]')
        str_birth = year_element.text
            # ojo el string debe ser el selector 
        birth_year = str_birth.split()
        year_birthday = birth_year[4] # esto es lo que tenemos que guardar como dato en la columna birthday
    except:
        year_birthday = "Not found"
        
        #Peliculas por las que es conocido/a
    films_done = []
    all_films_done = driver.find_elements("css selector", '.ipc-primary-image-list-card__title')
        
    for i in range(len(all_films_done)):
        films_done.append(all_films_done[i].get_attribute('innerHTML')) 

    # Qué trabajos hace
    jobs = []
    all_jobs = driver.find_elements("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li')
    for i in range(len(all_jobs)):
        jobs.append(all_jobs[i].get_attribute('innerHTML')) 

    #Premios
    awards = 0
    try: 
        awards_element = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul/li/span')
        awards = awards_element.text
        # Expresión regular para encontrar hasta 4 dígitos consecutivos, que sería el nº de premios
        pattern = r'\b\d{1,4}\b'
        # Buscar todos los dígitos en el string
        matches = re.findall(pattern, awards)
        # Imprimir cada coincidencia
        for match in matches:
            awards = match
            break

    except:
        awards = 0

    informacion_actor = {
        "actor_id": actor_id, 
        "name": name,
        "year_birthday": year_birthday,
        "films_done": films_done,
        "jobs": jobs,
        "awards": awards
        }
    list_info_actors.append(informacion_actor)
        

Extrayendo la informacion del actor nm0893341
Extrayendo la informacion del actor nm0590452
Extrayendo la informacion del actor nm0000901
Extrayendo la informacion del actor nm0197950
Extrayendo la informacion del actor nm0033929
Extrayendo la informacion del actor nm0289842
Extrayendo la informacion del actor nm0617741
Extrayendo la informacion del actor nm0472020
Extrayendo la informacion del actor nm0449166
Extrayendo la informacion del actor nm0773723
Extrayendo la informacion del actor nm0593192
Extrayendo la informacion del actor nm0527017
Extrayendo la informacion del actor nm0110917
Extrayendo la informacion del actor nm0859183
Extrayendo la informacion del actor nm0288031
Extrayendo la informacion del actor nm0486742
Extrayendo la informacion del actor nm0275987
Extrayendo la informacion del actor nm0129478
Extrayendo la informacion del actor nm0408990
Extrayendo la informacion del actor nm0038066
Extrayendo la informacion del actor nm0473228
Extrayendo la informacion del acto

In [34]:
#Comprobamos que se ha hecho la lista
print(list_info_actors )

[{'actor_id': 'nm0893341', 'name': 'Lino Ventura', 'year_birthday': '1919', 'films_done': ['El ejército de las sombras', 'Ascensor para el cadalso', 'Una dama y un bribón', 'Los miserables'], 'jobs': ['Reparto', 'Banda sonora'], 'awards': '3'}, {'actor_id': 'nm0590452', 'name': 'Sandra Milo', 'year_birthday': '1933', 'films_done': ['Fellini 8½', 'Giulietta de los espíritus', 'Adua y sus amigas', 'La entrevista'], 'jobs': ['Reparto', 'Equipos adicionales'], 'awards': '6'}, {'actor_id': 'nm0000901', 'name': 'Jean-Paul Belmondo', 'year_birthday': '1933', 'films_done': ['Pierrot, el loco', 'Léon Morin, sacerdote', 'Al final de la escapada', 'Casino Royale'], 'jobs': ['Reparto', 'Producción', 'Especialistas'], 'awards': '6'}, {'actor_id': 'nm0197950', 'name': 'Marcel Dalio', 'year_birthday': '1899', 'films_done': ['Sabrina', 'La regla del juego', 'Tener y no tener', 'Cómo robar un millón y...'], 'jobs': ['Reparto', 'Equipos adicionales', 'Guion'], 'awards': 0}, {'actor_id': 'nm0033929', 'na

In [35]:
#Convertimos los datos en un DataFrame
df_info_actors = pd.DataFrame(list_info_actors)

In [36]:
#Comprobamos el DataFrame
df_info_actors

,actor_id,name,year_birthday,films_done,jobs,awards
0,nm0893341,Lino Ventura,1919,"[El ejército de las sombras, Ascensor para el ...","[Reparto, Banda sonora]",3
1,nm0590452,Sandra Milo,1933,"[Fellini 8½, Giulietta de los espíritus, Adua ...","[Reparto, Equipos adicionales]",6
2,nm0000901,Jean-Paul Belmondo,1933,"[Pierrot, el loco, Léon Morin, sacerdote, Al f...","[Reparto, Producción, Especialistas]",6
3,nm0197950,Marcel Dalio,1899,"[Sabrina, La regla del juego, Tener y no tener...","[Reparto, Equipos adicionales, Guion]",0
4,nm0033929,Michel Ardan,Not found,"[Les bidasses en vadrouille, La grande nouba, ...","[Reparto, Producción, Guion]",0
...,...,...,...,...,...,...
515,nm0058384,John Bartha,1915,"[El bueno, el feo y el malo, 001, operación Ca...",[Reparto],0
516,nm0049673,Raf Baldassarre,1932,"[El gran silencio, 001, operación Caribe, Los ...",[Reparto],Raf Baldasar
517,nm0029549,Nando Angelini,Not found,"[001, operación Caribe, Supersiete llama al Ca...","[Reparto, Dirección de la segunda unidad o asi...",Nick Angel
518,nm0452471,Wolfgang Kieling,1924,"[Cortina rasgada, Polizeirevier Davidswache, J...","[Reparto, Equipos adicionales, Dirección de la...",4


In [37]:
#Convertimos en CSV
df_info_actors.to_csv("actors.csv",index=False)